# Project: Predicting Housing Prices in King County

We will use the **King County House Sales dataset**:  
[House Sales Prediction (Kaggle)](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction)

# Data Wrangling

- Data Types
    - Date is formatted as an ISO string. Created new int columns for month and year, then dropped the original date column
    - No other object columns
    - Several ordinal columns are already stored as int
    - id and zipcode are categorical columns stored as int. Will use these as is.
- Unstructured Data
    - No text fields that needed cleanup or preprocessing.
- Missing or Duplicate Data
    - No missing data found
    - No duplicate rows found
    - Duplicate ids were found - meaning the same house was sold twice within the data timeframe. Only the newest sale event was kept
- Outliers
    - Dropped one row explicitly where bedrooms value was 33, but was normally distributed otherwise. Suspect that this is a typo of '3'.
    - I implemented a sigma-filtering mechanism for continuous columns (defined as n_unique > 20) where the rows is dropped when any column exceeds the x-sigma threshold. I started initially with 10*sigma filtering, then increase the filtering as part of the study.
    - Ordinal columns are handled manually based on data behavior.
        - Grade (1-13) appears to have a monotonic polynomial relationship to price. No data handling.
        - View (0-4) appears to have a monotonic relationship to price. No data handling.
        - Waterfront (0-1). No data handling.
        - Condition (1-5). No data handling.
        - Bedrooms (0-11) became sparse and non-linear above 8, so I winsorized to 8+.
        - Bathrooms is a hybrid continuous-ordinal column due to 'half-bathrooms'. With a max of 8, it still has n_unique > 20 and would default to continuous treatment and sigma filtering (but no data removed at 10-sigma threshold). I opted to apply a floor function and winsorized to 6+, which helped with sparse data buckets and had a monotonic relationship to price.
        - Bedrooms and Bathrooms both contained rows with 0. Zero bedrooms could be a studio condo, but 0 bathrooms seems impossible for a dwelling. Perhaps these represent sales of empty lots. In the end, dropping these rows made MAE and RMSE scores increase, so I opted to leave these rows in the data.
        - Floors is convoluted by basements (which adds 0.5). Once removed (using floor function), the column is still not well-behaved: 3 story houses are a preference or forced by lot size, and not a driver of price. I used floor and winsorizeing to change this data into a 0-1 binary called 'multi-story'
- yr_renovated was over-populated with zeros for houses that had not been renovated.
    - combined yr_built and yr_renovated into a single column.
    - converted yr columns into age at time of sale.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('kc_house_data.csv')

df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
# Check for data type cleanup
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [4]:
# Check for duplicate rows
print(' Duplicate Rows: \n')
print(df.duplicated().value_counts())
print('\n Duplicate id: \n')
print(df['id'].duplicated().value_counts())

 Duplicate Rows: 

False    21613
Name: count, dtype: int64

 Duplicate id: 

id
False    21436
True       177
Name: count, dtype: int64


In [5]:
# Check for missing data
df.isnull().sum()                    #none

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [6]:
# date column cleanup.

df['year'] = df['date'].str[:4].astype('int')
df['month'] = df['date'].str[4:6].astype('int')
df['date'] = pd.to_datetime(df['date'])

In [7]:
# create age columns

df['yr_built_or_reno'] = np.select([(df['yr_renovated'] > 0), (df['yr_renovated'] == 0)],
                                          [df['yr_renovated'], df['yr_built']])

df['age_built_or_reno'] = df['year'] - df['yr_built_or_reno']

In [8]:
# drop duplicates (keep newest)

df = df.sort_values(['id', 'date'], ascending=[True,False])
df = df.drop_duplicates(subset=["id"])

# manual data filtering
df = df.drop(df[df['bedrooms'] == 33].index)

#drop date column - no longer needed
df = df.drop(columns=['date'])

In [9]:
# implement winsorized columns

df['bedrooms_win'] = df['bedrooms'].where(df['bedrooms'] < 8, other=8)
df['bathrooms_win'] = np.floor(df['bathrooms'].where(df['bathrooms'] < 6, other=6)).astype('int')
df['multi_story'] = (np.floor(df['floors'].where(df['floors'] < 2, other=2)) - 1).astype('int')

In [10]:
# Describe stats
stats_df = df.describe().round(2).reset_index()
stats_df

,index,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,long,sqft_living15,sqft_lot15,year,month,yr_built_or_reno,age_built_or_reno,bedrooms_win,bathrooms_win,multi_story
0,count,2.143500e+04,21435.00,21435.00,21435.00,21435.00,21435.00,21435.00,21435.00,21435.00,...,21435.00,21435.00,21435.00,21435.00,21435.00,21435.00,21435.00,21435.00,21435.00,21435.00
1,mean,4.580867e+09,541645.37,3.37,2.12,2082.73,15136.06,1.50,0.01,0.24,...,-122.21,1988.35,12786.34,2014.33,6.56,1973.49,40.83,3.37,1.75,0.42
2,std,2.876618e+09,367322.88,0.91,0.77,919.16,41539.54,0.54,0.09,0.77,...,0.14,685.70,27376.05,0.47,3.12,28.81,28.82,0.90,0.73,0.49
3,min,1.000102e+06,75000.00,0.00,0.00,290.00,520.00,1.00,0.00,0.00,...,-122.52,399.00,651.00,2014.00,1.00,1900.00,-1.00,0.00,0.00,0.00
4,25%,2.123700e+09,324844.00,3.00,1.75,1430.00,5040.00,1.00,0.00,0.00,...,-122.33,1490.00,5100.00,2014.00,4.00,1954.00,15.00,3.00,1.00,0.00
5,50%,3.904921e+09,450000.00,3.00,2.25,1920.00,7614.00,1.50,0.00,0.00,...,-122.23,1840.00,7620.00,2014.00,6.00,1978.00,37.00,3.00,2.00,0.00
6,75%,7.308750e+09,645000.00,4.00,2.50,2550.00,10696.50,2.00,0.00,0.00,...,-122.12,2370.00,10087.50,2015.00,9.00,1999.00,60.00,4.00,2.00,1.00
7,max,9.900000e+09,7700000.00,11.00,8.00,13540.00,1651359.00,3.50,1.00,4.00,...,-121.32,6210.00,871200.00,2015.00,12.00,2015.00,115.00,8.00,6.00,1.00


In [11]:
# Sorted table of correlations to price

price_correlations = df.corr().abs()['price'].round(3).reset_index()
price_correlations = price_correlations.sort_values('price', ascending=False)
price_correlations.columns = ['column', 'corr']

price_correlations

,column,corr
1,price,1.000
4,sqft_living,0.701
10,grade,0.666
11,sqft_above,0.605
18,sqft_living15,0.584
3,bathrooms,0.524
25,bathrooms_win,0.507
8,view,0.397
12,sqft_basement,0.324
24,bedrooms_win,0.317
